In [ ]:
import pandas as pd
import json
import re



df = pd.read_csv("/content/clean.csv")

def extract_state(eligibility, description):
    
    text = f"{eligibility} {description}".lower()

    states = [
        "andhra pradesh", "arunachal pradesh", "assam", "bihar", "chhattisgarh", "goa", "gujarat",
        "haryana", "himachal pradesh", "jharkhand", "karnataka", "kerala", "madhya pradesh",
        "maharashtra", "manipur", "meghalaya", "mizoram", "nagaland", "odisha", "punjab",
        "rajasthan", "sikkim", "tamil nadu", "telangana", "tripura", "uttar pradesh",
        "uttarakhand", "west bengal", "andaman and nicobar", "chandigarh", "dadra and nagar haveli",
        "daman and diu", "delhi", "jammu and kashmir", "ladakh", "lakshadweep", "puducherry"
    ]

    if "north eastern region" in text:
        return "North Eastern Region"
    if any(keyword in text for keyword in ["government of india", "ministry of", "national"]):
        for state in states:
            if state in text or f"government of {state}" in text or f"resident of {state}" in text:
                return state.title()
        return "National"
    for state in states:
        if state in text or f"government of {state}" in text or f"resident of {state}" in text:
            return state.title()

    return "Not specified"

def create_context(row):
    scheme_name = row['Scheme Name']
    description = row['Description & Benefits'] if pd.notna(row['Description & Benefits']) else ""
    eligibility = row['Eligibility Criteria'] if pd.notna(row['Eligibility Criteria']) else ""
    application = row['Application Process'] if pd.notna(row['Application Process']) else "Not specified"

    # Truncate description and eligibility to avoid excessive length
    description = description[:500] + ("..." if len(description) > 500 else "")
    eligibility = eligibility[:500] + ("..." if len(eligibility) > 500 else "")

    context = f"The {scheme_name} aims to {description.lower()} Eligible candidates include {eligibility.lower()} The application process is: {application.lower()}."
    return context

# Process the CSV and create document objects
documents = []

for index, row in df.iterrows():
    # Extract metadata
    scheme_name = row['Scheme Name']
    tags = row['Tags'].split(", ") if pd.notna(row['Tags']) else []
    state = extract_state(row['Eligibility Criteria'], row['Description & Benefits'])
    source_url = "Not available"  # Default, as no URL is provided

    # Create context
    context = create_context(row)

    # Create document object
    doc = {
        "context": context,
        "metadata": {
            "scheme_name": scheme_name,
            "state": state,
            "tags": tags,
            "source_url": source_url
        }
    }

    documents.append(doc)

# Save or print the documents (for demonstration, printing first few)
print(json.dumps(documents[:3], indent=2))

# Optionally, save to a JSON file
with open("scheme_documents.json", "w") as f:
    json.dump(documents, f, indent=2)


[
  {
    "context": "The Biponi Scheme aims to the objective of the scheme is to support to the micro  small enterprises to participate in different trade fairs and events within the state in the country and abroad for marketing of their products and also getting the exposure. Eligible candidates include eligibility 1. the location of the industry must be in rural areas of assam. 1. the industry must be already existing and also the unit necessity is to have a continuous operation for a minimum of 2 years. 1. the industryunit should hold its own workshed for the process and production. 1. the age of the applicant should be not less than 18 years and more than 50 years to apply for this scheme. 1. there is no bar of educational qualification. The application process is: na.",
    "metadata": {
      "scheme_name": "Biponi Scheme",
      "state": "Assam",
      "tags": [
        "Biponi",
        "Business",
        "Entrepreneurship",
        "Micro"
      ],
      "source_url": "Not a

In [3]:
from sentence_transformers import SentenceTransformer
import json

def generate_embeddings(documents, model_name='BAAI/bge-small-en-v1.5', batch_size=32):
    """
    Generate embeddings for the context field of documents and add to metadata.

    Args:
        documents (list): List of dictionaries, each with 'context' and 'metadata' fields.
        model_name (str): Name of the SentenceTransformer model (default: BAAI/bge-small-en-v1.5).
        batch_size (int): Batch size for encoding (default: 32).

    Returns:
        list: Updated documents with embeddings added to metadata['context_embedding'].
    """
    # Initialize the model
    model = SentenceTransformer(model_name)

    # Extract contexts for batch encoding
    contexts = [doc['context'] for doc in documents]

    # Generate embeddings
    embeddings = model.encode(contexts, batch_size=batch_size, normalize_embeddings=True)

    # Add embeddings to metadata
    for doc, embedding in zip(documents, embeddings):
        doc['metadata']['context_embedding'] = embedding.tolist()  # Convert to list for JSON

    return documents

# Load JSON file
input_json_path = "/content/scheme_documents.json"  # Replace with your JSON file path
with open(input_json_path, 'r') as f:
    documents = json.load(f)

# Generate embeddings
updated_documents = generate_embeddings(documents)

# Save updated documents to a new JSON file
output_json_path = "scheme_documents_with_embeddings.json"
with open(output_json_path, 'w') as f:
    json.dump(updated_documents, f, indent=2)

# Print first document for verification
print(json.dumps(updated_documents[0], indent=2))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{
  "context": "The Biponi Scheme aims to the objective of the scheme is to support to the micro  small enterprises to participate in different trade fairs and events within the state in the country and abroad for marketing of their products and also getting the exposure. Eligible candidates include eligibility 1. the location of the industry must be in rural areas of assam. 1. the industry must be already existing and also the unit necessity is to have a continuous operation for a minimum of 2 years. 1. the industryunit should hold its own workshed for the process and production. 1. the age of the applicant should be not less than 18 years and more than 50 years to apply for this scheme. 1. there is no bar of educational qualification. The application process is: na.",
  "metadata": {
    "scheme_name": "Biponi Scheme",
    "state": "Assam",
    "tags": [
      "Biponi",
      "Business",
      "Entrepreneurship",
      "Micro"
    ],
    "source_url": "Not available",
    "context_em

In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 74.1 MB/s eta 0:00:00


In [5]:
from sentence_transformers import SentenceTransformer
import json
import faiss
import numpy as np

def generate_embeddings(documents, model_name='BAAI/bge-small-en-v1.5', batch_size=32):
    """
    Generate embeddings for the context field of documents and add to metadata.

    Args:
        documents (list): List of dictionaries, each with 'context' and 'metadata' fields.
        model_name (str): Name of the SentenceTransformer model (default: BAAI/bge-small-en-v1.5).
        batch_size (int): Batch size for encoding (default: 32).

    Returns:
        list: Updated documents with embeddings added to metadata['context_embedding'].
    """
    model = SentenceTransformer(model_name)
    contexts = [doc['context'] for doc in documents]
    embeddings = model.encode(contexts, batch_size=batch_size, normalize_embeddings=True)
    for doc, embedding in zip(documents, embeddings):
        doc['metadata']['context_embedding'] = embedding.tolist()
    return documents

def create_vector_space(documents, dimension=384):
    """
    Create a FAISS vector space for cosine similarity search.

    Args:
        documents (list): List of document objects with embeddings in metadata['context_embedding'].
        dimension (int): Dimension of the embeddings (default: 384 for BGE-Small).

    Returns:
        faiss.IndexFlatIP: FAISS index for cosine similarity search.
    """
    embeddings = np.array([doc['metadata']['context_embedding'] for doc in documents], dtype=np.float32)
    index = faiss.IndexFlatIP(dimension)  # Inner Product for cosine similarity (normalized embeddings)
    index.add(embeddings)
    return index

def search_vector_space(query, documents, index, model, k=5):
    """
    Search the vector space for the top-k most similar documents to the query.

    Args:
        query (str): Query string to search for.
        documents (list): List of document objects.
        index (faiss.IndexFlatIP): FAISS index with document embeddings.
        model (SentenceTransformer): Model to encode the query.
        k (int): Number of top results to return (default: 5).

    Returns:
        list: List of (document, similarity_score) tuples for top-k results.
    """
    # Encode the query
    query_embedding = model.encode([query], normalize_embeddings=True).astype(np.float32)

    # Search the index
    distances, indices = index.search(query_embedding, k)

    # Collect results
    results = []
    for idx, distance in zip(indices[0], distances[0]):
        if idx >= 0 and idx < len(documents):  # Ensure valid index
            results.append((documents[idx], float(distance)))  # Distance is cosine similarity

    return results

# Main processing
if __name__ == "__main__":
    # Load JSON file
    input_json_path = "/content/scheme_documents.json"  # Replace with your JSON file path
    with open(input_json_path, 'r') as f:
        documents = json.load(f)

    # Generate embeddings
    model = SentenceTransformer('BAAI/bge-small-en-v1.5')
    documents = generate_embeddings(documents)

    # Create vector space
    vector_space_index = create_vector_space(documents)

    # Save updated documents
    output_json_path = "scheme_documents_with_embeddings.json"
    with open(output_json_path, 'w') as f:
        json.dump(documents, f, indent=2)

    # Save FAISS index
    faiss.write_index(vector_space_index, "scheme_embeddings.faiss")

    # Example search
    query = "Support for small businesses in Assam"
    top_k_results = search_vector_space(query, documents, vector_space_index, model, k=3)

    # Print search results
    print("Top 3 similar documents for query:", query)
    for doc, score in top_k_results:
        print(f"\nSimilarity Score: {score:.4f}")
        print(f"Scheme Name: {doc['metadata']['scheme_name']}")
        print(f"Context: {doc['context'][:200]}...")  # Truncated for brevity

Top 3 similar documents for query: Support for small businesses in Assam

Similarity Score: 0.7500
Scheme Name: Biponi Scheme
Context: The Biponi Scheme aims to the objective of the scheme is to support to the micro  small enterprises to participate in different trade fairs and events within the state in the country and abroad for ma...

Similarity Score: 0.7478
Scheme Name: Market Support Scheme for Smaller Business Units
Context: The Market Support Scheme for Smaller Business Units aims to the objective of the market support scheme for smaller business units is to provide market support to the eligible it units in goa. a reimb...

Similarity Score: 0.7320
Scheme Name: Banglashree for Micro, Small and Medium Enterprises: Interest Subsidy on Term Loan (IS)
Context: The Banglashree for Micro, Small and Medium Enterprises: Interest Subsidy on Term Loan (IS) aims to interest subsidy on term loan is incentives under banglashree for msmes scheme implemented by the ms...


In [34]:
# 1. Install required packages
!pip install -q transformers torch sentence-transformers faiss-cpu

# 2. Full implementation with all dependencies
import json
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import pipeline

def retrieve_chunks(query, json_path, faiss_path, model_name='BAAI/bge-small-en-v1.5', k=3):
    """Retrieve relevant document chunks"""
    json_path = "/content/scheme_documents_with_embeddings.json"
    with open(json_path, 'r') as f:
        documents = json.load(f)

    index = faiss.read_index("/content/scheme_embeddings.faiss")
    model = SentenceTransformer(model_name)

    query_embedding = model.encode([query], normalize_embeddings=True).astype(np.float32)
    distances, indices = index.search(query_embedding, k)

    return [
        documents[idx] for idx in indices[0]
        if 0 <= idx < len(documents)
    ]

def augment_response(system_context, query, json_path, faiss_path, max_context_length=512):
    """Augment query with retrieved context"""
    # Retrieve documents
    documents = retrieve_chunks(
        query=query,
        json_path=json_path,
        faiss_path=faiss_path
    )

    # Build context string
    context = "\n".join([d["context"] for d in documents])[:max_context_length]

    return f"{system_context}\n\nContext:\n{context}\n\nQuestion: {query}"

def generate_response(system_context, query, json_path, faiss_path):
    """Generate final response using local model"""
    # Create augmented prompt
    prompt = augment_response(system_context, query, json_path, faiss_path)

    # Initialize model
    generator = pipeline(
        'text2text-generation',
        model='google/flan-t5-base',
        device=0 if torch.cuda.is_available() else -1,
        torch_dtype=torch.float16
    )

    # Generate response
    return generator(
        prompt,
        max_length=500,
        do_sample=True,
        temperature=0.7
    )[0]['generated_text']

if __name__ == "__main__":
    # Update these paths to match your actual file locations
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a helpful assistant providing accurate information about government schemes.
Answer using the context below. Keep responses under 3 sentences.if i ask abou schemes then give me schemes only. if i ask about eligiblity criteria then tell me about eligibility criteria only if i ask about the application process then tell me about application process only if i ask about ministries/departments give me ministries/department only , filter base don tags and context"""

    query = "Support for small businesses in Assam"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Response:", response)

Device set to use cuda:0


Query: Support for small businesses in Assam
Response: The Biponi Scheme aims to the objective of the scheme is to support to the micro small enterprises to participate in different trade fairs and events within the state in the country and abroad for marketing of their products and also getting the exposure


In [18]:
if __name__ == "__main__":
    # Update these paths to match your actual file locations
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a helpful assistant providing accurate information about government schemes.
Answer using the context below. Keep responses under 3 sentences."""

    query = "Scheme for disabled students in kerala"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Response:", response)

Device set to use cuda:0


Query: Scheme for disabled students in kerala
Response: The Financial Assistance To Disabled Students Pursuing (10th, 11th, 12th Equivalent Exams


In [20]:
if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "what schemes are for farmers of maharashtra"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: what schemes are for farmers of maharashtra
Complete Response: Bhausaheb Fundkar Falbag Lagvad Scheme


In [190]:
if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "what is Ministrie of Vivah Sahayata Yojana"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Device set to use cuda:0


Query: what is Ministrie of Vivah Sahayata Yojana
Complete Response: Vivah Sahayata Yojana is a scheme by the department of labour employment training skill development govt of jharkhand


In [191]:
if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "Application process for The Financial Assistance To Disabled Students Pursuing (10th, 11th, 12th Equivalent Exams"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: Application process for The Financial Assistance To Disabled Students Pursuing (10th, 11th, 12th Equivalent Exams
Complete Response: the applicant needs to visit the nearest anganwadi centre or district social justice office and collect the application form. the application form can also be downloaded from the of the social justice department government of kerala. the applicant needs to fill out the application form completely the application may be submitted to the district social justice officers dsjos or the concerned anganwadi centre.


In [ ]:
if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "what is Ministrie of Vivah Sahayata Yojana"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Device set to use cuda:0


Query: what is Ministrie of Vivah Sahayata Yojana
Complete Response: a scheme by the department of labour employment training skill development govt of jharkhand


In [ ]:
if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "what schemes are for farmers of maharashtra"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: what schemes are for farmers of maharashtra
Complete Response: Bhausaheb Fundkar Falbag Lagvad Scheme


In [ ]:
if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "Application process for The Financial Assistance To Disabled Students Pursuing (10th, 11th, 12th Equivalent Exams"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: Application process for The Financial Assistance To Disabled Students Pursuing (10th, 11th, 12th Equivalent Exams
Complete Response: The application process is: to avail of the benefits of the scheme the applicant needs to visit the nearest anganwadi centre or district social justice office and collect the application form.


In [12]:
if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "What is the objective of the biponi scheme?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What is the objective of the biponi scheme?
Complete Response: to support to the micro small enterprises to participate in different trade fairs and events within the state in the country and abroad for marketing of their products and also getting the exposure


In [195]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "Who is eligible to apply for the Mukhya Mantri Mahila Kisan Sashaktikaran Yojana?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: Who is eligible to apply for the Mukhya Mantri Mahila Kisan Sashaktikaran Yojana?
Complete Response: women farmers


In [17]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "What is the objective of the Chief Minister Floriculture Mission??"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What is the objective of the Chief Minister Floriculture Mission??
Complete Response: allround development of floriculture sector


In [18]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "What benefits are provided under the Chief Ministergi Shotharabasingi Tengbang??"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What benefits are provided under the Chief Ministergi Shotharabasingi Tengbang??
Complete Response: free travel concession to the persons with disabilities


In [19]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "What is the purpose of the Homes For Intellectually Impaired Persons scheme??"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What is the purpose of the Homes For Intellectually Impaired Persons scheme??
Complete Response: mentally deficient children who are in need of care and protection are admitted into shelter homes through the child welfare committees


In [21]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "What is the objective of the Distribution of Free House Sites to Scheduled Caste/Other Economically Backward Classes?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What is the objective of the Distribution of Free House Sites to Scheduled Caste/Other Economically Backward Classes?
Complete Response: to provide housing sites to enable them to construct their own houses


In [22]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "What is the purpose of the Scheme for Margin Money Grant to Nano Units?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What is the purpose of the Scheme for Margin Money Grant to Nano Units?
Complete Response: provide financial assistance to nano units in the state engaged in manufacturingjob work service activity having any type of value addition


In [23]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "Which scheme promotes natural farming in rajasthan?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: Which scheme promotes natural farming in rajasthan?
Complete Response: zero budget natual farming


In [26]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "Which scheme provides pensions for visually impaired persons in Tripura?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: Which scheme provides pensions for visually impaired persons in Tripura?
Complete Response: The Pension to the Persons who lost 100% eye sight


In [35]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "Which scheme supports girl children of jute mill workers?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: Which scheme supports girl children of jute mill workers?
Complete Response: The Scholarship Scheme for Girl Children of Jute Mills / MSMEs Workers


In [196]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "What are the eligibility criteria for the KSCSTE Post-Doctoral Fellowship Programme?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Token indices sequence length is longer than the specified maximum sequence length for this model (517 > 512). Running this sequence through the model will result in indexing errors
Device set to use cuda:0


Query: What are the eligibility criteria for the KSCSTE Post-Doctoral Fellowship Programme?
Complete Response: 1. applicant should be a permanent resident of kerala 2. applicant should have scored a minimum of 60 marks for postgraduation 3. applicant should have qualified ph.d. in scienceengineering subjects 4. applicant should not be a permanent employee at the time of application for the scheme. 5. age limit 40 years relaxed for 3 years for obc and 5 years for scst candidate 6. those who have availed postdoctoral fellowship earlier from kscste or elsewhere are not eligible to apply for kscste postdoctoral fellowships


In [197]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "Which scheme provides financial assistance for Goan Maand culture??"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: Which scheme provides financial assistance for Goan Maand culture??
Complete Response: The Financial Assistance to Revive, Preserve and Protect Goan Maand Culture


In [198]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "What are the eligibility criteria for the Performance Linked Grant Scheme for Smaller Business Units?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What are the eligibility criteria for the Performance Linked Grant Scheme for Smaller Business Units?
Complete Response: 1. the applicant should be a smaller business unit. 2. the applicant should have their operations in goa 3. the applicant should be one of the following proprietorship firm private limited company incorporated under the companies act 2013 or equivalent public limited company incorporated under the companies act 2013 or equivalent limited liability partnership incorporated under the limited liability partne...


In [47]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "Which department oversees the Leprosy Pension Scheme in Uttar Pradesh?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: Which department oversees the Leprosy Pension Scheme in Uttar Pradesh?
Complete Response: department of empowerment of persons with disabilities


In [50]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below."""

    query = "What is the objective of thr Financial Assistance for Marriage of Poor Scheduled Caste Brides?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What is the objective of thr Financial Assistance for Marriage of Poor Scheduled Caste Brides?
Complete Response: provide financial assistance to poor brides living below the poverty line to meet their marriage expenses


In [199]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below. if i am asking aligibility criteria give me eligibility criteria of the scheme from the context """

    query = "What are the eligibility criteria for the Homes For Intellectually Impaired Persons scheme??"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What are the eligibility criteria for the Homes For Intellectually Impaired Persons scheme??
Complete Response: 1. the applicant should be a citizen of india. 2. the applicant should be an orphan. 3. the applicant should be a mentally deficient child mdc who is in need of care and protection. 4. the percentage of the disability should be 40 or above.


In [200]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below. full eligibility criteria from context"""

    query = "What are the eligibility criteria for the NPC Students Internship Program?"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What are the eligibility criteria for the NPC Students Internship Program?
Complete Response: 1. a citizen of india 2. the applicant should not already have applied for the internship before in the current financial year 3. strong editing written and oral communication skills 4. strong interpersonal skills 5. strong proactive technical troubleshooting and problemsolving skills 6. strong research and networking 7. the applicant should be a citizen of india 8. the applicant should not already have applied for the internship before in the current financial year 9. the applicant should be a student pursuing graduation and postgraduation studies. 10. the applicant should be a student pursuing graduation and postgraduation studies. 11. education must be in a university or an institute


In [88]:

if __name__ == "__main__":
    json_path = "/content/scheme_documents_with_embeddings.json"
    faiss_path = "/content/scheme_embeddings.faiss"

    system_context = """You are a government scheme expert. Answer concisely using context below. exact application process"""

    query = "What is the application process for the Financial Incentives To Mothers Who Deliver A Girl Child (Mamta)"

    response = generate_response(system_context, query, json_path, faiss_path)

    print("Query:", query)
    print("Complete Response:", response)

Device set to use cuda:0


Query: What is the benefits for the Financial Incentives To Mothers Who Deliver A Girl Child (Mamta)
Complete Response: 10000 as a onetime financial incentive at the birth of the child. 1. those beneficiaries who had applied for the 1st installment of 5000 as on the date of this amended schem
